<a href="https://colab.research.google.com/github/LYZAcell/data_study/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%80%EB%84%90/%EB%8D%B0%EC%A0%80)%EC%9C%84%EA%B2%BD%EB%8F%84_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터저널 위경도 추출
- 노션 './전처리위경도전.csv' 업로드 후 시작

## 라이브러리 설치 및 import

In [1]:
pip install requests

In [7]:
import pandas as pd
import requests
import numpy as np

## 카카오맵 api 응답확인

In [3]:
# 카카오맵 호출되는지 확인
# api 키 권한오류 시 코랩, 로컬 등 링크 허용 필요

def test_address(address):
  # restapi키
  api_key = "ec7cbcebaedfbf302ed77da39ea7ef3b"
  headers = {"Authorization": f"KakaoAK {api_key}"}

  # API 호출 URL
  url = "https://dapi.kakao.com/v2/local/search/address.json"

  # 요청 파라미터
  params = {"query": address}

  # API 요청
  response = requests.get(url, headers=headers, params=params)

  if response.status_code == 200:
      result = response.json()
      if result['documents']:
          # 위도와 경도를 추출
          x = result['documents'][0]['x']  # 경도
          y = result['documents'][0]['y']  # 위도
          return x, y
      else:
          return None
  else:
      print(f"Error {response.status_code}: {response.text}")
      return None


In [5]:
# 사용 예제
address = "부산광역시 부산진구 전포대로 209번길 26"
coordinates = test_address(address)

if coordinates:
    print(f"주소: {address}")
    print(f"위도: {coordinates[1]}")
    print(f"경도: {coordinates[0]}")
else:
    print("좌표를 찾을 수 없습니다.")

주소: 부산광역시 부산진구 전포대로 209번길 26	
위도: 35.1563611729441
경도: 129.062888281786


## df 호출

In [6]:
origin_df = pd.read_csv('./전처리위경도전.csv', encoding = 'cp949')
df = origin_df.copy()
df.head()

,시도명,시군구명,폐교명,폐교연도,학교급구분명,활용현황구분명,건물연면적,소재지도로명주소,소재지지번주소
0,강원특별자치도,강릉시,왕산초 고단분교장,2010,초등학교,대부,771.00,NaN,강원특별자치도 강릉시 왕산면 고단리 32-18
1,강원특별자치도,강릉시,옥계초 남양분교장,2011,초등학교,대부,892.00,NaN,강원특별자치도 강릉시 옥계면 남양리 528
2,강원특별자치도,양양군,양양여자중학교,2019,중학교,자체활용,10566.93,NaN,강원특별자치도 양양군 양양읍 서문리 224
3,강원특별자치도,원주시,문막초 취병분교장,2014,초등학교,대부,539.58,NaN,강원특별자치도 원주시 문막읍 취병리 358
4,부산광역시,부산진구,중앙초등학교,2008,초등학교,자체활용,5178.00,부산광역시 부산진구 전포대로 209번길 26,NaN


In [13]:
def get_coordinates(address, api_key):
  #주소를 위경도로 변환하는 함수. if에 안걸릴 시 nan 반환
    headers = {"Authorization": f"KakaoAK {api_key}"}
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        print("위경도 응답 중")
        result = response.json()
        if result['documents']:
            x = result['documents'][0]['x']  # 경도
            y = result['documents'][0]['y']  # 위도
            return x, y
    return np.nan, np.nan

def get_address(row, api_key):
    #DataFrame의 행(row)에서 주소를 선택하고, 위경도를 반환하는 함수
    # 도로명주소와 지번주소 중 유효한 주소 선택
    if pd.notna(row['소재지도로명주소']):
        address = row['소재지도로명주소']
    elif pd.notna(row['소재지지번주소']):
        address = row['소재지지번주소']
    else:
        return np.nan, np.nan

    # 유효한 주소로 위경도 반환
    return get_coordinates(address, api_key)

In [14]:


# 반복문 시작
# 빈 리스트를 생성하여 경도와 위도 데이터를 저장할 준비
longitudes = []
latitudes = []

# DataFrame 순회
for index, row in df.iterrows():
    if pd.notna(row['소재지도로명주소']):
        address = row['소재지도로명주소']
    elif pd.notna(row['소재지지번주소']):
        address = row['소재지지번주소']
    else:
        longitudes.append(np.nan)
        latitudes.append(np.nan)
        continue

    longitude, latitude = get_coordinates(address, api_key)
    longitudes.append(longitude)
    latitudes.append(latitude)

# DataFrame에 경도와 위도 칼럼 추가
df['경도'] = longitudes
df['위도'] = latitudes

위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위경도 응답 중
위

KeyboardInterrupt: 

In [15]:
df

,시도명,시군구명,폐교명,폐교연도,학교급구분명,활용현황구분명,건물연면적,소재지도로명주소,소재지지번주소
0,강원특별자치도,강릉시,왕산초 고단분교장,2010,초등학교,대부,771.00,NaN,강원특별자치도 강릉시 왕산면 고단리 32-18
1,강원특별자치도,강릉시,옥계초 남양분교장,2011,초등학교,대부,892.00,NaN,강원특별자치도 강릉시 옥계면 남양리 528
2,강원특별자치도,양양군,양양여자중학교,2019,중학교,자체활용,10566.93,NaN,강원특별자치도 양양군 양양읍 서문리 224
3,강원특별자치도,원주시,문막초 취병분교장,2014,초등학교,대부,539.58,NaN,강원특별자치도 원주시 문막읍 취병리 358
4,부산광역시,부산진구,중앙초등학교,2008,초등학교,자체활용,5178.00,부산광역시 부산진구 전포대로 209번길 26,NaN
...,...,...,...,...,...,...,...,...,...
400,전라북도,무주군,삼방초,2008,초등학교,자체활용,670.69,전라북도 무주군 적상면 상비길 8-14,전라북도 무주군 적상면 삼유리 572
401,전라북도,장수군,원촌초,2010,초등학교,자체활용,988.72,전라북도 장수군 계북면 장무로 1326,전라북도 무주군 계북면 원촌리 1247
402,전라북도,순창군,구림중,2010,중학교,자체활용,2548.13,전라북도 순창군 구림면 연산2길 26-47,전라북도 순창군 구림면 운남리 52
403,전라북도,부안군,보안초,2010,초등학교,대부,1257.34,전라북도 부안군 보안면 선돌로 1060,전라북도 부안군 보안면 하입석리 573-2


In [26]:
# itterroa 순회
def get_coordinates(address, api_key):
    """주소를 위경도로 변환하는 함수"""
    headers = {"Authorization": f"KakaoAK {api_key}"}
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result['documents']:
            x = result['documents'][0]['x']  # 경도
            y = result['documents'][0]['y']  # 위도
            print(result,x,y)
            return x, y
    return np.nan, np.nan

In [28]:
#보안상 빼둠
api_key = "ec7cbcebaedfbf302ed77da39ea7ef3b"

# 좌표를 저장할 빈 리스트 초기화
longitudes = []
latitudes = []

# DataFrame 순회
for index, row in df.iterrows():
    if pd.notna(row['소재지도로명주소']):
        address = row['소재지도로명주소']
    elif pd.notna(row['소재지지번주소']):
        address = row['소재지지번주소']
    else:
        longitudes.append(np.nan)  # 주소를 찾을 수 없는 경우 NaN 추가
        latitudes.append(np.nan)
        continue  # 다음 행으로 이동

    longitude, latitude = get_coordinates(address, api_key)
    longitudes.append(longitude)
    latitudes.append(latitude)

# DataFrame에 경도 및 위도 열 추가
df['경도'] = longitudes  # 리스트를 새 열에 할당
df['위도'] = latitudes

{'documents': [{'address': {'address_name': '강원특별자치도 강릉시 왕산면 고단리 32-18', 'b_code': '5115032024', 'h_code': '5115032000', 'main_address_no': '32', 'mountain_yn': 'N', 'region_1depth_name': '강원특별자치도', 'region_2depth_name': '강릉시', 'region_3depth_h_name': '왕산면', 'region_3depth_name': '왕산면 고단리', 'sub_address_no': '18', 'x': '128.84865851311', 'y': '37.5199416538742'}, 'address_name': '강원특별자치도 강릉시 왕산면 고단리 32-18', 'address_type': 'REGION_ADDR', 'road_address': {'address_name': '강원특별자치도 강릉시 왕산면 왕산로 23-23', 'building_name': '왕산초등학교고단분교', 'main_building_no': '23', 'region_1depth_name': '강원특별자치도', 'region_2depth_name': '강릉시', 'region_3depth_name': '왕산면 고단리', 'road_name': '왕산로', 'sub_building_no': '23', 'underground_yn': 'N', 'x': '128.847803203531', 'y': '37.519895224153', 'zone_no': '25644'}, 'x': '128.84865851311', 'y': '37.5199416538742'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}} 128.84865851311 37.5199416538742
{'documents': [{'address': {'address_name': '강원특별자치도 강릉시 

KeyboardInterrupt: 

In [25]:
df

,시도명,시군구명,폐교명,폐교연도,학교급구분명,활용현황구분명,건물연면적,소재지도로명주소,소재지지번주소,경도,위도
0,강원특별자치도,강릉시,왕산초 고단분교장,2010,초등학교,대부,771.00,NaN,강원특별자치도 강릉시 왕산면 고단리 32-18,128.84865851311,37.5199416538742
1,강원특별자치도,강릉시,옥계초 남양분교장,2011,초등학교,대부,892.00,NaN,강원특별자치도 강릉시 옥계면 남양리 528,129.031198121156,37.5806003602398
2,강원특별자치도,양양군,양양여자중학교,2019,중학교,자체활용,10566.93,NaN,강원특별자치도 양양군 양양읍 서문리 224,128.613236978881,38.0728988633267
3,강원특별자치도,원주시,문막초 취병분교장,2014,초등학교,대부,539.58,NaN,강원특별자치도 원주시 문막읍 취병리 358,127.800600665593,37.3112064525129
4,부산광역시,부산진구,중앙초등학교,2008,초등학교,자체활용,5178.00,부산광역시 부산진구 전포대로 209번길 26,NaN,129.062888281786,35.1563611729441
...,...,...,...,...,...,...,...,...,...,...,...
400,전라북도,무주군,삼방초,2008,초등학교,자체활용,670.69,전라북도 무주군 적상면 상비길 8-14,전라북도 무주군 적상면 삼유리 572,127.613726765133,35.9232355412811
401,전라북도,장수군,원촌초,2010,초등학교,자체활용,988.72,전라북도 장수군 계북면 장무로 1326,전라북도 무주군 계북면 원촌리 1247,127.631040378065,35.8108961507907
402,전라북도,순창군,구림중,2010,중학교,자체활용,2548.13,전라북도 순창군 구림면 연산2길 26-47,전라북도 순창군 구림면 운남리 52,127.108442149463,35.4520160427029
403,전라북도,부안군,보안초,2010,초등학교,대부,1257.34,전라북도 부안군 보안면 선돌로 1060,전라북도 부안군 보안면 하입석리 573-2,126.690021099289,35.6221775066961
